
# ROS2_23_24_recuperacion -- ejercicio 1

¡IMPORTANTE! En cada ejercicio se especifica el nombre de los paquetes, ficheros de lanzamiento etc. Es imprescindible que sigáis estas instrucciones ya que se va a testear la entrega del examen automáticamente.

Utilizando ROS2 se desea implementar un paquete para cargar un mundo que contenga:

- el entorno office con el modelo una oficina

- el robot turtlebot_3

Para ello se proporciona un archivo comprimido (ej_01.zip) que contiene todos los archivos necesarios para cargar en gazebo el modelo office y el robot turtlebot3.

## Pasos a seguir

1. Crear el paquete ex_recu_24_preg_01  con dependencias: rclcpp,  gazebo_ros_pkgs,  geometry_msgs,  nav_msgs,  sensor_msgs,  tf2 y turtlebot3_gazebo

In [ ]:
# En visual abro la carpeta donde me pidan ponerlo y abro una terminal y pongo esto ahí para crear la carpeta
ros2 pkg create --build-type ament_cmake ex_recu_24_preg_01 --dependencies rclcpp gazebo_ros_pkgs geometry_msgs nav_msgs sensor_msgs tf2 turtlebot3_gazebo

2. Crear los siguientes directorios en el paquete anterior:

- models: con los modelos necesarios turtlebot3_burger y todos los modelos que hacen falta para cargar la librería (se proporcionan estos modelos en el directorio models del archivo ej01.zip)

- world: con el modelo del mundo (burger_office.world) que se proporciona en el archivo ej01.zip

- launch: contendrá el fichero de lanzamiento del mundo (lo crearemos en el apartado 3)

- src:  incluir los ficheros necesarios para implementar el comportamiento del robot en simulación ( turtlebot3_drive.cpp y turtlebot3_drive.hpp) que se proporcionan

- urdf: con el fichero de descripción hardware (turtlenbot3_burger.urdf) que se proporciona

3. Crear el fichero de lanzamiento paquete ex_recu_24_preg_01_tb3.launch.py  que lance el entorno proporcionado junto con el robot turtlebot3

In [ ]:
import os

from ament_index_python.packages import get_package_share_directory
from launch import LaunchDescription
from launch.actions import IncludeLaunchDescription, DeclareLaunchArgument
from launch.launch_description_sources import PythonLaunchDescriptionSource
from launch.substitutions import LaunchConfiguration
from launch_ros.substitutions import FindPackageShare
from launch_ros.actions import Node

TURTLEBOT3_MODEL = os.environ['TURTLEBOT3_MODEL']

def generate_launch_description():
    world_file_name = 'world/' + TURTLEBOT3_MODEL + '.model'
    urdf_file_name = 'urdf/turtlebot3_' + TURTLEBOT3_MODEL + '.urdf'
    pkg_share = FindPackageShare(package='my_world').find('my_world')
    gazebo_models_path = os.path.join(pkg_share, 'models')
    os.environ["GAZEBO_MODEL_PATH"] = gazebo_models_path
    use_sim_time = LaunchConfiguration('use_sim_time', default='true')
    world = os.path.join(pkg_share, world_file_name)
    urdf = os.path.join(pkg_share,  urdf_file_name )

    launch_file_dir = os.path.join(get_package_share_directory('turtlebot3_gazebo'), 'launch')
    pkg_gazebo_ros = get_package_share_directory('gazebo_ros')

    return LaunchDescription([
        IncludeLaunchDescription(
            PythonLaunchDescriptionSource(
                os.path.join(pkg_gazebo_ros, 'launch', 'gzserver.launch.py')
            ),
            launch_arguments={'world': world}.items(),
        ),

        IncludeLaunchDescription(
            PythonLaunchDescriptionSource(
                os.path.join(pkg_gazebo_ros, 'launch', 'gzclient.launch.py')
            ),
        ),

        IncludeLaunchDescription(
            PythonLaunchDescriptionSource([launch_file_dir, '/robot_state_publisher.launch.py']),
            launch_arguments={'use_sim_time': use_sim_time}.items(),
        ),

    ])

4. Hacer las modificaciones necesarias en el fichero CMakeLists.txt para que todo se ejecute correctamente

In [ ]:
################################################################################
# Set minimum required version of cmake, project name and compile options
################################################################################
cmake_minimum_required(VERSION 3.5)
project(my_world)

if(NOT CMAKE_CXX_STANDARD)
  set(CMAKE_CXX_STANDARD 14)
endif()
my_world
if(CMAKE_COMPILER_IS_GNUCXX OR CMAKE_CXX_COMPILER_ID MATCHES "Clang")
  add_compile_options(-Wall -Wextra -Wpedantic)
endif()

if(MSVC)
  add_compile_definitions(_USE_MATH_DEFINES)
endif()

################################################################################
# Find ament packages and libraries for ament and system dependencies
################################################################################
find_package(ament_cmake REQUIRED)
find_package(gazebo REQUIRED)
find_package(gazebo_ros_pkgs REQUIRED)
find_package(geometry_msgs REQUIRED)
find_package(nav_msgs REQUIRED)
find_package(rclcpp REQUIRED)
find_package(sensor_msgs REQUIRED)
find_package(tf2 REQUIRED)
find_package(turtlebot3_gazebo REQUIRED)

################################################################################
# Build
################################################################################
link_directories(
  ${GAZEBO_LIBRARY_DIRS}
)

include_directories(
  include
  ${GAZEBO_INCLUDE_DIRS}
)

set(dependencies
  "geometry_msgs"
  "nav_msgs"
  "rclcpp"
  "sensor_msgs"
  "tf2"
)

set(EXEC_NAME "turtlebot3_drive")

add_executable(${EXEC_NAME} src/turtlebot3_drive.cpp)
ament_target_dependencies(${EXEC_NAME} ${dependencies})

################################################################################
# Install
################################################################################
install(TARGETS ${EXEC_NAME}
  DESTINATION lib/${PROJECT_NAME}
)

install(DIRECTORY launch models world urdf
  DESTINATION share/${PROJECT_NAME}/
)

install(DIRECTORY include/
  DESTINATION include/
)

################################################################################
# Macro for ament package
################################################################################
ament_export_include_directories(include)
ament_export_dependencies(gazebo_ros_pkgs)
ament_export_dependencies(geometry_msgs)
ament_export_dependencies(nav_msgs)
ament_export_dependencies(rclcpp)
ament_export_dependencies(sensor_msgs)
ament_export_dependencies(tf2)
ament_package()

5. Compilar y simular

In [ ]:
cd turtlebot3_ws
colcon build --packages-select ex_recu_24_preg_01
source install/setup.bash

Antes de poner el launch asegurarnos de que estamos en la ruta correcta


Modelo Asun en home
```
export GAZEBO_MODEL_PATH=$HOME/examen_ros2/src/ex_recu_24_preg_01/models:$GAZEBO_MODEL_PATH
```

Para mi ejemplo usé este
```
gazebo ~/examen_ros2/ex_recu_24_preg_01/world/burger_office.world --verbose
```

In [ ]:
ros2 launch ex_recu_24_preg_01 ex_recu_24_preg_01_tb3.launch.py  #usar el comando anterior, este da error

6. Ejecutar el visualizador RVIZ y configurarlo con los siguientes displays:
- Modelo del Robot
- Laser
- Transformadas (TF)

 Entregar un pantallazo que muestre claramente la vista del visualizador RVIZ con los displays seleccionados

In [ ]:
export TURTLEBOT3_MODEL=burger
ros2 launch turtlebot3_cartographer cartographer.launch.py use_sim_time:=True

Si queréis que aparezca el modelo del robot en el visualizador RVIZ debeis añadir (botón Add) el Topic RobotModel y cambiar dos parámetros:

Description Source: ponemos File

Description File: seleccionamos la ruta en la que se encuentra el fichero de descripción del robot, en nuestro caso está en turtlebot3_ws/src/turtlebot3/turtlebot3_description/urdf/turtlebot3_burger.urdf. En el Polilab este fichero está en /opt/ros/galactic/share/turtlebot3_description/urdf/turtlebot3_burger.urdf.

Revisar que todos se ha cargado correctamente.
Usar el menú File > Save Config As para guardar el archivo de configuración en el directorio rviz que acabamos de crear.

## rqt gui

In [ ]:
ros2 run rqt_gui rqt_gui

In [ ]:
rqt_graph

🟢 Nodos
/cartographer_node
Nodo principal del sistema de SLAM de Cartographer. Se encarga de construir el mapa y de publicar información sobre submapas y transformaciones.

/robot_state_publisher
Publica el estado actual del robot (poses de las articulaciones y frames) en el tópico /tf. Se basa en el archivo URDF del robot.

/occupancy_grid_node
Toma la información de submapas de Cartographer y genera un mapa tipo Occupancy Grid (rejilla de ocupación), útil para navegación.

/transform_listener_Impl_5637b08a6c40
Listener interno de TF que escucha las transformaciones en /tf. No lo lanzas directamente, es generado automáticamente cuando usas TF en tu código.

/gazebo
Nodo del simulador Gazebo. Probablemente esté corriendo el mundo simulado en el que se encuentra el robot.

🔷 Tópicos

/clock: Este tópico proporciona la hora del sistema y es especialmente útil cuando estás corriendo simulaciones con Gazebo o reproduciendo bags, ya que sincroniza los tiempos.

/constraint_list: Publica un MarkerArray para visualizar las restricciones entre nodos en el grafo de optimización del SLAM, útil para ver cómo Cartographer está ajustando el mapa.

/joint_states: Publica los estados de las articulaciones del robot, como posiciones y velocidades. Este mensaje es importante para robots con múltiples grados de libertad.

/landmark_poses_list: Publica las posiciones de marcadores o "landmarks" que Cartographer detecta o utiliza como referencias fijas para mejorar la localización.

/map: Contiene el mapa en formato de rejilla de ocupación (OccupancyGrid) generado por el sistema SLAM. Este tópico se suele usar con herramientas de navegación y visualización como RViz.

/odom: Publica la odometría del robot, es decir, su posición estimada a partir de sensores de movimiento (como encoders). Se puede usar como fuente de movimiento base.

/parameter_events: Publica eventos relacionados con cambios en parámetros dinámicos dentro del sistema ROS2. Es parte del sistema de configuración de nodos.

/performance_metrics: Proviene de Gazebo y proporciona métricas sobre el rendimiento de la simulación, como uso de CPU o tiempo de simulación real vs. tiempo de simulación.

/robot_description: Contiene el modelo URDF del robot en forma de cadena de texto. Este modelo describe su estructura, articulaciones y enlaces.

/rosout: Tópico donde se publican los mensajes de log de los nodos ROS. Funciona como la consola del sistema distribuido.

/scan: Publica los datos del sensor LIDAR o escáner láser en formato LaserScan. Este mensaje es esencial para SLAM y navegación basada en percepción.

/scan_matched_points2: Publica una nube de puntos (PointCloud2) generada a partir del proceso de scan matching en Cartographer. Refleja los puntos ajustados a los submapas.

/submap_list: Contiene una lista de los submapas que Cartographer ha generado hasta el momento. Es clave para visualizar y entender cómo se está construyendo el mapa general.

/tf: Publica transformaciones dinámicas entre frames del robot, necesarias para entender la relación espacial entre sensores, enlaces del robot y el entorno.

/tf_static: Similar a /tf, pero solo para transformaciones que no cambian en el tiempo. Por ejemplo, la posición relativa fija entre dos componentes.

/trajectory_node_list: Publica un MarkerArray que representa gráficamente la trayectoria del robot (los nodos de trayectoria) que Cartographer ha calculado. Muy útil para depurar o visualizar el recorrido del robot en RViz.

----

🧩 Nodos
/cartographer_node
Realiza SLAM combinando los datos del LiDAR y la odometría para generar el mapa y las transformaciones entre marcos.

/occupancy_grid_node
Convierte los submapas de Cartographer en un mapa de ocupación para navegación.

/turtlebot3_diff_drive
Controla el movimiento del robot y publica odometría.

/turtlebot3_laserscan
Nodo que publica los datos del sensor láser del robot.

/turtlebot3_imu
Publica los datos del sensor inercial (IMU).

/turtlebot3_joint_state
Publica el estado de las articulaciones del robot.

/tf y /tf_static
Publican transformaciones entre los distintos marcos de coordenadas del sistema.

/transform_listener_impl_*
Nodos que escuchan y utilizan transformaciones de tf.



📡 Tópicos
/scan
Datos del LiDAR, enviados como mensajes de tipo LaserScan.

/odom
Publica la odometría del robot (posición y orientación estimada).

/imu
Datos del sensor inercial (acelerómetro, giroscopio).

/joint_states
Información sobre las posiciones y velocidades de las ruedas del robot.

/scan_matched_points2
Nube de puntos generada por Cartographer a partir del escáner láser.

/submap_list
Lista de submapas que Cartographer utiliza para construir el mapa completo.

/map
Mapa de ocupación final, usado para navegación.

/tf y /tf_static
Publican las transformaciones entre marcos de coordenadas.

/clicked_point, /cmd_vel, /initialpose, /goal
Tópicos usados desde RViz para interacción con el robot (establecer objetivos, poses iniciales y enviar comandos de movimiento).

/trajectory_node_list, /landmark_poses_list, /constraint_list
Tópicos usados por Cartographer para visualizar restricciones, trayectorias y landmarks durante el SLAM.




